# Robotics Project 1

In this project, we will solve the position and velocity level forward kinematics of the NeXCoM 6-DoF miniBoT and optionally verify that our solution matches with the output of its
implementation in MuJoCo.

## Question 1 

The coordinate frames were assigned and Denavit-Hartenberg parameters chosen as seen below. 

![Coordinate-frame-diagram](coordDiagram.png)




In [ ]:
# Import stuff and define some functions. Other code I wrote is imported from python files. 

import mujoco as mj
from pathlib import Path
import numpy as np
import mujoco.viewer
import time
from spatialmath import SE3, SO3
from spatialmath.base import tr2adjoint
from numpy import deg2rad

import importlib
import kinematics
importlib.reload(kinematics)# make sure any changes I make are updated immediately. by forcing a reimport to avoid caching. 
from kinematics import Kinematics

def get_pose(data, body_id):
    """
    Extracts the position and rotation of a body from the mujoco data structure.
    """
    #Extract the positions
    rot = data.xmat[body_id].reshape(3,3)
    pos = data.xpos[body_id] * 1000 #Convert to mm


    #Convert to transformation matrix
    transformation = np.eye(4) 
    transformation[:3, :3] = rot  
    transformation[:3, 3] = pos 
    return SE3(transformation)

def geometric_inverse(desired_pose: np.ndarray)-> list:
    """
    Given a desired pose, return the joint angles that achieve it.
    Uses hardcoded values and geometric methods to find the joint angles. 
    """
    #get the position of the wrist center. d
    desired_translation = desired_pose[3,:3]
    desired_rotation = desired_pose[:3, :3]
    wrist_center = desired_translation - 95 * desired_rotation @ np.array([0, 0, 1])

    d = 27.5
    r = np.sqrt(wrist_center[0]**2 + wrist_center[1]**2) - d# Projected distance from joint 2 to wrist center in x0 y0 plane. 
    s = wrist_center[2] - 339
    theta1 = np.arctan2(wrist_center[1], wrist_center[0])

    # Now we need to find theta 2 and 3. These are independent of theta1
    
    a2 = 250
    a3 = np.sqrt(70**2 + 250**2)
    extra_theta3 = np.pi - np.arctan(250/70)



    #We find theta3 using the law of cosines. 
    theta3 = np.arccos((r**2 + s**2 - a2**2 - a3**2) / (-2 * a2 * a3)) - extra_theta3

    print(theta3)
    theta2a = np.arctan2(s, r) - np.arctan2(a3 * np.sin(theta3), a2 + a3 * np.cos(theta3))
    print(theta2a)
    # theta2b = np.arctan2(s, r) - np.arctan2(a3 * np.sin(theta3b), a2 + a3 * np.cos(theta3b))

    return [theta1, theta2a, theta3, 0, 0, 0]

# Helps with printouts. 
np.set_printoptions(suppress=True)

# Load the mujoco model for answer verification. 
xml_path = Path("CAD") / "robot_model.xml"
model = mj.MjModel.from_xml_path(str(xml_path))
mujoco_model_data = mj.MjData(model)


end_effector_body_name="end-effector"
end_effector_body_id = model.body(name=end_effector_body_name).id


# Define the paramaters for this robot arm. 
dh_table = [[True, 27.5, np.pi/2, 339],
            [True, 250, 0, 0],
            [True, 70, np.pi/2, 0],
            [True, 0, -np.pi/2, 250],
            [True, 0, np.pi/2, 0],
            [True, 0, 0, 95]
            ]


home_angles = np.array([0, np.pi/2, 0, 0, 0, 0])
# Create a kinematics object with the home angles and the DH table.
mini_bot_kinematics = Kinematics(home_angles, dh_table)
# Compute the transformation of a known position for use later. 
home_pos = mini_bot_kinematics.foreward(home_angles)


### (a) [30 points] Forward Kinematics Problem

Solve the **position-level forward kinematics** problem. Use this solution to find the end-effector pose **ξ** given that the joint angles are:

$$\mathbf{\theta}=\begin{bmatrix}
0° & 90° & 0° & 0° & -90° & 0°
\end{bmatrix}$$


In [30]:
question_1_angles = np.array([0, deg2rad(90), 0, 0, deg2rad(-90), 0]) 

print("Question 1a transformation from foreward kinematics:")
question_one_transformation = mini_bot_kinematics.foreward(question_1_angles)
# Print the transformation to easy viewing. 
print(question_one_transformation)
# Print the answer as asked by the project instructions. 
print(mini_bot_kinematics.transformation_to_minimal_representation(question_one_transformation))
    

Question 1a transformation from foreward kinematics:
   1         0         0         277.5     
   0        -1         0         0         
   0         0        -1         564       
   0         0         0         1         

[277.5  -0.  564.  180.   -0.    0. ]


### (b) Inverse position kinematics
...

In [31]:
given_transformation = np.array([[.7551, .4013, .5184, 399.1255], 
                                 [.6084, -.7235, -.3262, 171.01526], 
                                 [.2441, .5617, -.7905, 416.0308], 
                                 [0, 0, 0, 1]])

given_transformation = mini_bot_kinematics.foreward(joint_angles=home_angles).A
asdf = geometric_inverse(given_transformation)
print(asdf)

-0.35338909741308844
-1.1941883988669042
[np.float64(3.141592653589793), np.float64(-1.1941883988669042), np.float64(-0.35338909741308844), 0, 0, 0]


## Question 2-Velocity level kinematics

Here the kinematic Jacobian matrix must be computed for the manipulator at the joint angles found from question 1b. 

Explain how this is done. 

The jacobian matrix is defined as

$$\mathbf{\eta}=\mathbf{J}\mathbf{\dot{q}}$$

rearranging we get
$$\mathbf{J}^{-1}\mathbf{\eta}=\mathbf{\dot{q}}$$

The jacobian is not always invertible but because this robot is 6dof it will always be square. According to chatGPT, a 6x6 jacobian is invertible as long as the robot is not at a singularity, so for our purposes, it should be a safe assumption. 

For now this will be done at the home position because I haven't found the position of question 1b yet. 

In [32]:
np.set_printoptions(suppress=True)
question_2_twist = np.array([2, -1, .5, 100, -200, -300])
question_2_angles = home_angles
home_jacobian = mini_bot_kinematics.jacobian(joint_angles=question_2_angles, link=6)
# twist = home_jacobian @ goal_rates
# print(twist)
rates = np.linalg.pinv(mini_bot_kinematics.jacobian(joint_angles=question_2_angles, link=6)) @ question_2_twist
print(rates)

mj.mj_resetData(model, mujoco_model_data)
mujoco_model_data.qpos[:len(question_2_angles)] = question_2_angles

mujoco_model_data.qvel[:len(rates)] = rates

mj.mj_forward(model, mujoco_model_data)


# Tried and true code from project 1
# Body twist: measured in the end-effector's (body) frame [linear; angular]
body_lin_vel = mujoco_model_data.sensordata[0 : 3]
body_ang_vel = mujoco_model_data.sensordata[3 : 6]
measured_twist = np.concatenate([body_lin_vel, body_ang_vel])


measured_twist[:3] = get_pose(mujoco_model_data, end_effector_body_id).R @ measured_twist[:3] * 1000 # Convert linear units to mm
measured_twist[3:] = get_pose(mujoco_model_data, end_effector_body_id).R @ measured_twist[3:]
print(f"Measured Twist: {measured_twist}")

[ -0.00484659  21.27144    -97.26944     50.         275.998
  50.        ]
Measured Twist: [   2.           -1.80535612    0.5         100.         -200.
   -0.00484659]


## Question 3 - mujoco verification
In this question I will use the mujoco model of this robot to verify the answers to the rest of the project. 

### Question 1 Foreword position kinematics

For part A, the foreword kinematics we can simply set the model to the joint angles given and read off the transformation of the end-effector. Just because I will also verify the transformation to the home position. 


In [33]:
# Home position
mujoco_model_data.qpos[:len(home_angles)] = home_angles
mj.mj_forward(model, mujoco_model_data)
mujoco_home_pose = get_pose(mujoco_model_data, end_effector_body_id)

if np.allclose(np.array(mujoco_home_pose), np.array(home_pos), atol=1e-8):
    print("The home position was correctly computed. ")
else:
    print("The home position was not correctly computed.")

# Question 1 part A
mujoco_model_data.qpos[:len(question_1_angles)] = question_1_angles
mj.mj_forward(model, mujoco_model_data)
mujoco_question_one_pose = get_pose(mujoco_model_data, end_effector_body_id)

if np.allclose(np.array(mujoco_question_one_pose), np.array(question_one_transformation), atol=1e-8):
    print("Question 1 part A is correct.")
else:
    print("Question 1 part A is not correct. ")

The home position was correctly computed. 
Question 1 part A is correct.


### Question 1 B-Inverse position kinematics

In [34]:
pass

### Question 2 Velocity Level Kinematics.    

Here we will check if the jacobian function works correctly by setting  joint positions and velocities in mujoco, reading the twist, and comparing it to what my function yields, the same way as in project 1. 


In [35]:
mj.mj_resetData(model, mujoco_model_data)

test_angles = home_angles
mujoco_model_data.qpos[:len(test_angles)] = test_angles

# Use random rotation rates each test. 
test_rot_rates = np.random.uniform(low=-1, high=1, size=6)
# test_rot_rates = np.array([1, 0, 0, 0, 0, 0])
mujoco_model_data.qvel[:len(test_rot_rates)] = test_rot_rates


mj.mj_forward(model, mujoco_model_data)


# Tried and true code from project 1
# Body twist: measured in the end-effector's (body) frame [linear; angular]
body_lin_vel = mujoco_model_data.sensordata[0 : 3]
body_ang_vel = mujoco_model_data.sensordata[3 : 6]
measured_twist = np.concatenate([body_lin_vel, body_ang_vel])


measured_twist[:3] = get_pose(mujoco_model_data, end_effector_body_id).R @ measured_twist[:3] * 1000 # Convert linear units to mm
measured_twist[3:] = get_pose(mujoco_model_data, end_effector_body_id).R @ measured_twist[3:]

test_jacobian = mini_bot_kinematics.jacobian(joint_angles=test_angles, link=6)
analytic_twist = test_jacobian @ test_rot_rates

if np.allclose(measured_twist, analytic_twist, atol=1e-8):
    print("Twist was correct")
else:
    print("Twist was incorrect. ")
    print(f"Measured: {measured_twist}")
    print(f"Analytic: {analytic_twist}")

Twist was correct
